In [16]:
import os
import time
import yaml
from llama_index.llms import Replicate

with open('config/config.yml', 'r') as file:
    config = yaml.safe_load(file)

os.environ["REPLICATE_API_TOKEN"] = config['API_KEY']

llm = Replicate(
    model=config['replicate_config']['model'],
    temperature=config['replicate_config']['temperature'],
    additional_kwargs=config['replicate_config']['additional_kwargs']
)

# set tokenizer to match LLM
from llama_index import set_global_tokenizer
from transformers import AutoTokenizer

set_global_tokenizer(
    AutoTokenizer.from_pretrained(config['tokenizer']).encode
)

from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import ServiceContext

embed_model = HuggingFaceEmbedding(model_name=config['embed_model'])
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

query = "Share one amazing motivational quote. End it with a newline character. Don't generate any introductory line."
query_engine = index.as_query_engine()
response = query_engine.query(query)
res_text = response.response.split('\n')
res_text

[' "You are a helpful assistant."',
 '"Farewell, my master, always in every chance first in my mind, as you deserve to be."',
 '"My master, see I am not asleep, and I compel myself to sleep, that you may not be angry with me."',
 '"Gratia and I will have to fight for it; I doubt I shall not get the better of her."',
 '"Whenas, by God\'s mercy you shall stand upright, my spirit too will stand firm."',
 '"To grow together like fellow branches in matter of good correspondence and affection; but not in matter of opinions."',
 '"It is not possible that any nature should be inferior unto art, since that all arts imitate nature."',
 '"Justice cannot be preserved, if either we settle our minds and affections upon worldly things; or be apt to be deceived, or rash, and inconstant."']

In [17]:
query = "Share one amazing motivational quote. End it with a newline character. Don't generate any introductory line."
query_engine = index.as_query_engine()
response = query_engine.query(query)
res_text = response.response.split('\n')
res_text

[' "Farewell, my master, always in every chance first in my mind, as you deserve to be. My master, see I am not asleep, and I compel myself to sleep, that you may not be angry with me." - Marcus Aurelius']